<!-- Assignment 2 - SS 2023 -->

# Vision Networks and Fast Training (15 points)

This notebook contains one of the assignments for the exercises in Deep Learning and Neural Nets 2.
It provides a skeleton, i.e. code with gaps, that will be filled out by you in different exercises.
All exercise descriptions are visually annotated by a vertical bar on the left and some extra indentation,
unless you already messed with your jupyter notebook configuration.
Any questions that are not part of the exercise statement do not need to be answered,
but should rather be interpreted as triggers to guide your thought process.

**Note**: The cells in the introductory part (before the first subtitle)
perform all necessary imports and provide utility functions that should work without (too much) problems.
Please, do not alter this code or add extra import statements in your submission, unless explicitly allowed!

<span style="color:#d95c4c">**IMPORTANT:**</span> Please, change the name of your submission file so that it contains your student ID!

In this assignment, we will take a closer look at some famous vision architectures.
Since most of these architectures are very large, it requires high-end hardware to train from scratch.
To leverage the limited availability of hardware, also *Transfer Learning* can be used.
By using stored weights of a large network a new network can be trained cheaply on new datasets.

In [28]:
import inspect
import torch
import torchvision
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from tqdm.notebook import trange

torch.manual_seed(1806)
torch.cuda.manual_seed(1806)

In [ ]:
# google colab data management
import os.path

try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    _home = 'gdrive/MyDrive/'
except ImportError:
    _home = '~'
finally:
    data_root = os.path.join(_home, '.pytorch')

print(data_root)

Mounted at /content/gdrive
gdrive/MyDrive/.pytorch


## LeNet-5 and its Offspring

![LeNet-5 architecture](https://miro.medium.com/max/2154/1*1TI1aGBZ4dybR6__DI9dzA.png)

The LeNet-5 architecture (depicted above) is one of the first convolutional networks.
Since convolutions are extremely well suited for many computer vision tasks,
a wide variety of network architectures using convolutional layers has become available.
Although the differences in performance are sometimes large,
the architectures can generally be considered variations on the same theme.

### Alexnet

![alex-net architecture](https://cdn-images-1.medium.com/max/1000/1*wzflNwJw9QkjWWvTosXhNw.png)

In 2012 Alex Krizhevsky et al. won the [Imagenet Large Scale Visual Recognition Challenge](http://www.image-net.org/challenges/LSVRC/) (ILSVRC).
The network they used, which is known as *Alex-net*, is depicted below and follows the same basic principles as LeNet-5.
Alex-net has quite a bit more parameters than LeNet-5, therefore it requires a large amount of computational resources to train.

To speed up training time, Alex-net was trained on GPU.
Since GPUs have access to little memory compared to CPUs (especially back in the days),
alex-net did not fit on a single GPU and required 2 GPUs to train the model,
hence the distinction between two paths in the illustration of the network.

On modern GPUs, it is no longer a problem to fit alex-net on a single GPU.
Due to the fact that deep learning frameworks mostly support hardware acceleration,
it has even become extremely easy and almost common to train (large) networks on GPUs.
A more detailed description on how to achieve this in pytorch, is given below.

Another important add-on, is the use of the dropout regularisation technique in the fully connected layers.
From DL & NN 1 you should remember that dropout behaves differently during testing and training.
When using Dropout or other modules with different behaviour, e.g. BatchNorm, in pytorch,
it is important to make sure that your network operates in the right mode.
To do this, the `nn.Module` class provides the `train` and `eval` methods
and invokes it on all submodules to assure that the desired behaviour is triggered.

### Pytorch GPU acceleration

In pytorch, training a model on GPU is relatively easy.
To copy a tensor `x` from main memory (or wherever it may be) to GPU memory,
all we need to do is call `x.to('cuda')` or equivalently `x.cuda()`.
When multiple GPUs are available, `x.to('cuda:0')` copies a tensor to the first GPU,
`x.to('cuda:1')` to the second, etc.
Similarly, to copy a tensor from a GPU (or again wherever it may be) to main memory,
`x.to('cpu')` or equivalently `x.cpu()` can be used.

Whenever a computation is done on tensors that reside on a specific device,
the result will also be on that device.
It is not possible, however, to make computations with tensors from different devices.
This means that the training of an entire network automatically takes place on e.g. a GPU,
as soon as all the variables reside on the same device.
When working with neural networks,
this is the case if both the network parameters and the data are moved to the same device.

To move all parameters of a network to the correct device,
The `nn.Module` class provides a convenience `to` method
that moves all registered parameters, buffers and submodules to the correct device.

As for the data, it is often possible to fit the entire dataset in GPU memory.
However, often it does not provide any advantages or it even comes with disadvantages.
E.g. the `MNIST` dataset from `torchvision` provides PIL images that can not reside on GPU.
If the dataset would be stored on the GPU, the data would have to move to CPU first,
where the pre-processing is done on the PIL images, and then move back to the GPU.
Therefore, it is considered good practice to keep the dataset in main memory
and move the samples to the GPU only when they are needed for computation.

### Exercise 1: Hardware Acceleration (3 points)

In order to allow our computations to be accelerated,
the utility functions `evaluate` and `update` require some minor adjustments.

 > Alter the `evaluate` and `update` functions from assignment 1
 > so that it is assured that the inputs are on the same device as the network parameters.
 > Also put the networks in the right modes so that dropout etc. work correctly.

In [11]:
@torch.no_grad()
def evaluate(network: nn.Module, data: DataLoader, metric: callable) -> list:
    results = []
    network.to('cuda')
    for x, y in data:
        x = x.to('cuda')
        y = y.to('cuda')
        logits = network(x)
        res = metric(logits, y)
        results.append(res.item())
    return results


@torch.enable_grad()
def update(network: nn.Module, data: DataLoader, loss: nn.Module,
           opt: optim.Optimizer) -> list:
    network.to('cuda')
    errs = []
    for x, y in data:
        x = x.to('cuda')
        y = y.to('cuda')
        logits = network(x)
        err = loss(logits, y)
        errs.append(err.item())
        opt.zero_grad()
        err.backward()
        opt.step()
    return errs

In [12]:
# sanity check
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        return self.fc(x)

x_data = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0]])
y_data = torch.tensor([[1.0], [2.0], [3.0], [4.0]])
dataset = torch.utils.data.TensorDataset(x_data, y_data)
data_loader = torch.utils.data.DataLoader(dataset, shuffle=True)
model = SimpleModel()
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
errs = update(model, data_loader, loss_fn, optimizer)
eval_errs = evaluate(model, data_loader, loss_fn)
assert len(errs) == 4; assert len(eval_errs) == 4

In [13]:
# sanity check
class HackyLoss(nn.Module):

    def __init__(self):
        super().__init__()
        self.devices = []

    def forward(self, logits, y):
        self.devices.append(logits.device)
        return torch.nn.functional.cross_entropy(logits, y)


test_net = nn.Linear(512, 10).train().to("cpu")
test_data = torch.utils.data.TensorDataset(
    torch.randn(1, 512), torch.randint(10, size=(1, ))
)
test_loss = HackyLoss()

test_loss.devices = []
test_net.to("cuda")
try:
    evaluate(test_net, DataLoader(test_data), test_loss)
except RuntimeError:
    pass

assert len(test_loss.devices) > 0, (
    "ex1: evaluate function does not work on cuda device"
)
assert all(dev.type == "cuda" for dev in test_loss.devices), (
    "ex1: evaluate function does not use cuda device"
)

In [14]:
# Test Cell: do not edit or delete!

In [15]:
# Test Cell: do not edit or delete!

In [16]:
# Test Cell: do not edit or delete!

In [17]:
# Test Cell: do not edit or delete!

In [18]:
# Test Cell: do not edit or delete!

In [19]:
# Test Cell: do not edit or delete!

### VGG

<img src="https://miro.medium.com/max/2628/1*lZTWFT36PXsZZK3HjZ3jFQ.png"
     alt="VGG architecture" style="width: 70%; margin: auto" />

The Visual Geometry Group at Oxford University introduced
different versions of architectures that are now known as VGG net.
11-layer, 16-layer and 19-layer variants exist,
all of which use only 3x3 convolutions in the feature extraction part.

After winning the Imagenet Large Scale Visual Recognition Challenge (ILSVRC) in 2014,
the weights of the winning models were made [available](https://www.robots.ox.ac.uk/~vgg/research/very_deep/).
This made it possible for researchers with lower computational budgets
to make use of the features the network has extracted for natural images.
Since 2021, large pre-trained models often end up serving as [foundation models](https://en.wikipedia.org/wiki/Foundation_models).
Note that VGG is a very small model compared to modern "large" models today.

### Exercise 2: VGG for CIFAR-10 (2 points)

Most vision architectures have been trained on the ImageNet dataset, which is hard to come by:
it is very large (a few 100GB) and requires registration to get access to the images.
[CIFAR-10 and CIFAR-100](https://www.cs.toronto.edu/~kriz/cifar.html)
are similar datasets that are much easier to obtain
and they are one of the standard datasets in `torchvision.datasets`.
In this exercise the goal is to modify a vision network that was trained on ImageNet
to make predictions on CIFAR-10 so that we can reuse large parts of the weights.

 > Create a network with the same feature extraction architecture as
 > `torchvision.models.VGG` so that it can be used for CIFAR images.
 > Concretely, the goal is to replace the classifier to predict CIFAR labels
 > instead of the Imagenet labels.
 > Use global average pooling to make the classifier independent of the exact image size.
 > Keep the classifier architecture rectangular, i.e. same width for all layers (except for the classes).

**Hint:** Take a look at [`torchvision.models.VGG`](https://pytorch.org/vision/0.13/_modules/torchvision/models/vgg.html) if you need some inspiration.

In [47]:
class CifarVGG(nn.Module):
    """ Variant of the VGG network for classifying CIFAR images. """

    def __init__(self, features: nn.Module, num_classes: int = 10):
        """
        Parameters
        ----------
        features : nn.Module
            The convolutional part of the VGG network.
        num_classes : int
            The number of output classes in the data.
        """
        # YOUR CODE HERE
        super(CifarVGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 512),
            nn.ReLU(True),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x



In [48]:
# sanity check
test_net = CifarVGG(nn.Identity()).eval().requires_grad_(False)
try:
    ref = test_net(torch.zeros(1, 512, 7, 7))
except RuntimeError:
    raise AssertionError("ex2: classifier does not seem to work with default image sizes") from None

assert torch.all(test_net(torch.zeros(1, 512, 14, 14)) == ref), (
    "ex2: architecture depends on input image size, did you forget global average pooling?")

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

### Exercise 3: Existing Features (2 points)

Training a network like VGG (or any of the other networks in this assignment)
can take a few hours when training on a GPU.
Therefore it is often useful to be able to load pre-trained weights into the network.
Also, saving a model that has been trained for hours can often save a lot of time.
In pytorch this is possible through what is called
[`state_dict`s](https://pytorch.org/tutorials/beginner/saving_loading_models.html).
Saving the parameters of a pytorch module can be done with `torch.save(module.state_dict(), path)`,
whereas loading saved parameters is done with `module.load_state_dict(torch.load(path))`.

 > Write a function `vgg_init_` to initialise a `CifarVGG` network.
 > It should load the pre-trained weights for the **11-layer variant of VGG** from `torchvision.models.vgg`
 > to initialise the feature extractor of the model
 > and reasonably initialise the classifier using initialisation functions from `torch.nn.init`.

**Hint:** you can use all of the functions available in `torchvision.models.vgg`.

In [49]:
def vgg_init_(network: CifarVGG):
    """
    Initialise a CifarVGG network with a pre-trained VGG feature extractor.

    Parameters
    ----------
    network : CifarVGG
        The model to initialise.
    """
    from torchvision.models import vgg
    # YOUR CODE HERE
    vgg_model = vgg.vgg11(pretrained = True)
    model.features.load_state_dict(vgg_model.features.state_dict())

    #initialize the params for the network
    for module in network.modules():
        if isinstance(module, nn.Conv2d):
          nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity = 'relu')
          if module.bias is not None:
            nn.init.zeros_(module.bias)



In [50]:
# sanity check
vgg11 = torchvision.models.vgg11()
network = CifarVGG(vgg11.features, num_classes=10)
vgg_init_(network)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth
100%|██████████| 507M/507M [00:07<00:00, 68.1MB/s]


AttributeError: 'SimpleModel' object has no attribute 'features'

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

### Exercise 4: Training (part of) the Network (4 points)

Obviously, a classifier for CIFAR 10 will be different from a classifier for Imagenet.
With the initialisation above, the `CifarVGG` has a ready-to-go feature extractor,
but the classifier part still has to be trained.
To do this training efficiently, there are a few things left to do.

 > The code below should train the entire network on the entire dataset for a few epochs.
 > Modify the code so that
 > 1. it only trains the classifier part of the network using SGD
 > and leaves the convolutional feature extractor untouched, i.e., *frozen*.
 > 2. the 32x32 CIFAR images are upscaled to 224x224 pixels.
 > 3. training is done on the GPU, which is generally faster.
 > 4. it uses a parallel dataloader to make sure that the GPU does not have to wait for data.
 > 5. only a random subset of 500 images from the CIFAR data is used for training.
 > 6. a random subset of 500 images from the CIFAR data is used as validation data.
 > You will also need to include a round of validation in the training loop.
 > This should give you some confidence that the classifier is learning something useful.

**Hint:** you might find useful tools under
[`torch.utils.data`](https://pytorch.org/docs/stable/data.html).

In [ ]:
def get_cifar10(root: str, batch_size: int = 32, resize: tuple[int, int] = (224, 224),
                num_train: int = 500, num_valid: int = 500, num_workers: int = 4):
    """
    Get dataloader(s) for CIFAR-10.

    Parameters
    ----------
    root : str
        Path to directory where CIFAR-10 dataset is stored.
    batch_size : int, optional
        The number of samples per mini-batch.
    resize : tuple of int, optional
        Desired width and height of loaded images.
    num_train : int, optional
        Number of (random) samples to use for training.
    num_valid : int, optional
        Number of (random) samples to use for validation.
    num_workers : int, optional
        Number of parallel processes to use for loading data.

    Returns
    -------
    train_batches : DataLoader
        A dataloader that loads mini-batches of CIFAR-10 for training.
    valid_batches : DataLoader
        A dataloader that loads (mini-)batches of CIFAR-10 for validation.
    """
    normalise=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    ])
    cifar10 = torchvision.datasets.CIFAR10(root, transform=normalise, download=True)
    train_batches = DataLoader(cifar10, batch_size=batch_size)
    # YOUR CODE HERE
    raise NotImplementedError()
    return train_batches, None

In [ ]:
# Test Cell: do not edit or delete!
train_batches, valid_batches = get_cifar10(data_root)

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
def get_vgg(num_classes: int = 10, device: str = "cpu"):
    """
    Create and initialise VGG network on given device.

    Parameters
    ----------
    num_classes : int, optional
        The number of output units for the network.
    device : str, optional
        A string representing the device to work on
    """
    vgg11 = torchvision.models.vgg11()
    net = CifarVGG(vgg11.features, num_classes=10)
    vgg_init_(net)
    # YOUR CODE HERE
    raise NotImplementedError()
    return net

In [ ]:
# Test Cell: do not edit or delete!
network = get_vgg(device='cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class TransferTrainer:

    def __init__(self, model: nn.Module):
        """
        Create a trainer for transfer learning.

        Parameters
        ----------
        model : nn.Module
            The model to train.
        """
        self.model = model
        self.objective = nn.CrossEntropyLoss(reduction="sum")
        self.optimiser = None
        # YOUR CODE HERE
        raise NotImplementedError()

    def train(self, train_batches: DataLoader, valid_batches: DataLoader,
              num_epochs: int = 1):
        """
        Train (part of) the network for a number of epochs.

        Parameters
        ----------
        train_batches : DataLoader
            The training data for updating the network.
        valid_batches : DataLoader
            The validation data for evaluating the network.
        num_epochs : int, optional
            The number of iterations over the training data.
        """
        train_errs, valid_errs = [], []
        for _ in trange(num_epochs):
            local_errs = update(self.model, train_batches, self.objective, self.optimiser)
            train_errs.append(sum(local_errs) / len(local_errs) / train_batches.batch_size)
            # YOUR CODE HERE
            raise NotImplementedError()

        return train_errs, valid_errs


In [ ]:
trainer = TransferTrainer(network)
train_errs, valid_errs = trainer.train(train_batches, valid_batches, 20)

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# plot learning curves
from matplotlib import pyplot as plt
plt.plot(range(1, len(train_errs) + 1), train_errs, label="train")
plt.plot(range(1, len(valid_errs) + 1), valid_errs, label="valid")
plt.legend()
print(f"ran on {next(network.parameters()).device}")

## Skip-connections

One of the most popular modern network architectures for vision is the residual network.
The main feature of this architecture is the so-called skip-connection,
which allows to combine the activations with the original inputs in each layer.
Since these skip-connections open up a gradient highway,
they make it possible to train much deeper networks than is possible without the skip-connections.

Mathematically, the simplest form of a skip connection can be written as
$$\boldsymbol{s} = \boldsymbol{x} + f(\boldsymbol{x}).$$
In order for this to work, the dimensions of $\boldsymbol{x}$ and $f(\boldsymbol{x})$ must line up.
This means that in this formulation, only square layers,
i.e. layers with the same number of inputs and outputs, are possible.

In order to use a skip-connection on layers that reduce the dimensionality,
a linear transform on $\boldsymbol{x}$ can be inserted in the equation.
Since also other operations are possible, on both inputs and (pre-)activations,
we can generalise the skip-connection formula to
$$\boldsymbol{s} = \boldsymbol{C} \cdot \boldsymbol{x} + \boldsymbol{T} \cdot f(\boldsymbol{x}),$$
where $\boldsymbol{C}$ and $\boldsymbol{T}$ are linear transformations (a.k.a. matrices).

### Exercise 5: Pre-Residual Networks (4 points)

The original and most commonly used residual networks actually do not implement skip connections as in the formula above.
Upon closer inspection (e.g. `torchvision.models.resnet`), it becomes clear that the most famous skip-connection looks more like

$$\boldsymbol{a} = \phi(\boldsymbol{x} + f(\boldsymbol{x})),$$

where $\phi$ is some non-linear activation function.
This non-linearity typically interferes with the signal propagation of the network.
As a result, gradients might still vanish despite the skip-connection.

Pre-Residual Networks aim to counter this problem by moving skip-connections to the level of pre-activations, such that

$$\boldsymbol{a} = \boldsymbol{x} + f(\phi(\boldsymbol{x})).$$

This way, clean signal propagation can be guaranteed and learning should become easier.

 > Implement the `PreResBlock` class so that it can be used as a layer in a pre-residual network.
 > The residual part of the network, $f$, should be a small network with two convolutional layers.
 > Both layers should use the given `kernel_size` and preserve the image size if `stride` is one.
 > If `stride` is greater than one, the network should reduce the spatial dimensions by this factor.
 > Make sure that the network also works if `in_channels != out_channels` and `stride > 1`.
 > Try to avoid unnecessary parameters, especially for the skip-connection.
 > Precice information on the implementation of the `PreResBlock` can be found in this paper [Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027)

In [ ]:
class PreResBlock(nn.Module):
    """ Residual block using skip-connections on pre-activation level. """

    def __init__(self, in_channels: int, out_channels: int, kernel_size: int = 3,
                 stride: int = 1, phi: nn.Module = nn.ReLU(), extra_pars: bool = True):
        """
        Parameters
        ----------
        in_channels : int
            Number of input channels.
        out_channels : int
            Number of output channels.
        kernel_size : int
            Size of the kernel in all dimensions.
        stride : int
            Factor by which to reduce the spatial dimensions.
        phi : nn.Module
            The activation function to use in the residual branch.
        """
        super().__init__()
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# Test Cell: do not edit or delete!
block = PreResBlock(8, 8)
out = block(torch.zeros(1, 8, 32, 32))
sum(par.numel() for par in block.parameters()), out.shape

In [ ]:
# Test Cell: do not edit or delete!
strided_block = PreResBlock(8, 8, stride=2)
out = strided_block(torch.zeros(1, 8, 32, 32))
sum(par.numel() for par in strided_block.parameters()), out.shape

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!